In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
from pynq import Overlay
overlay = Overlay('./bitstream/LUT24bits.bit')

In [3]:
overlay.ip_dict

{'LUT24bits_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xaf85f6d0>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'LUT24bits_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S00_AXI',
  'parameters': {'C_S00_AXI_ADDR_WIDTH': '4',
   'C_S00_AXI_BASEADDR': '0x43C00000',
   'C_S00_AXI_DATA_WIDTH': '32',
   'C_S00_AXI_HIGHADDR': '0x43C0FFFF',
   'Component_Name': 'LUT24bits_LUT24bits_0_0',
   'EDK_IPTYPE': 'PERIPHERAL'},
  'phys_addr': 1136656384,
  'registers': {},
  'state': None,
  'type': 'xilinx.com:user:LUT24bits:2.0'}}

In [4]:
from pynq import MMIO
ip = MMIO(0x43C00000, 0x10000)

In [5]:
ip.device

### 將 r_LUT  的location輸出值轉為2補數:

In [6]:
def twos_complement_32bit(x):
    """將 32-bit 補數整數轉為有號值"""
    if x >= (1 << 31):
        return x - (1 << 32)
    else:
        return x

#### Test:

In [7]:
location_1 = -14 
remainder_1 = 11646

ip.write_mm(0, location_1)
ip.write_mm(8, remainder_1)


remainder = ip.read_mm(4)
location = ip.read_mm(12)


print(remainder)
print(twos_complement_32bit(location))

5645
-35


### 開始則中查表驗證

## 折衷查表:
### 用 Single Error Correction 解 Double Error Correction (A 取Double_AWE的 A)

#### 1.取 A的bits $\approx$ N的bits $\Rightarrow$ A = 18613  (30bits), N的範圍: 0 ~ 1073741823 (30bits)

### Define function
* (Not including 2's complement range

In [8]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value

## 把重複的 AWE 去除
def remove_duplicates(input_list):
    unique_list = []
    for number in input_list:
        if number not in unique_list:
            unique_list.append(number)
    return unique_list


def get_sign(number):
    return 1 if number > 0 else -1 if number < 0 else 0


print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


In [9]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data1 = (1 << 24) - 1    ## 
data2 = (1 << 24) - 19
data3 = (1 << 24) - 15
AN = 13837
input1 = data1*AN
input2 = data2*AN
input3 = data3*AN
bits1 = bits_needed(input1)
bits2 = bits_needed(input2)
bits3 = bits_needed(input3)
print("inputs1的位元數:", bits1)
print("inputs2的位元數:", bits2)
print("inputs3的位元數:", bits3)

inputs1的位元數: 38
inputs2的位元數: 38
inputs3的位元數: 38


### 產生1-bit AWE (Arithmetic-Weight Error)
* 並產生 Delta Q 的 list

In [10]:
AWE1_1=[]
e1_1=0     ## 錯誤數量
e1_data1=[]
h = [];   #位置
for i in range(0, bits1):
    e1_data1.insert(e1_1 , input1 + 2**i)
    AWE1_1.insert(e1_1, 2**i)
    e1_1 = e1_1 + 1
    e1_data1.insert(e1_1 , input1 - 2**i)
    AWE1_1.insert(e1_1, -2**i)
    e1_1 = e1_1 + 1
    h.insert(i,i+1)

AWE1_2=[]
e1_2=0     ## 錯誤數量
e1_data2=[]
for i in range(0, bits2):
    e1_data2.insert(e1_2 , input2 + 2**i)
    AWE1_2.insert(e1_2, 2**i)
    e1_2 = e1_2 + 1
    e1_data2.insert(e1_2 , input2 - 2**i)
    AWE1_2.insert(e1_2, -2**i)
    e1_2 = e1_2 + 1
    h.insert(i,i+1)
    
AWE1_3=[]
e1_3=0     ## 錯誤數量
e1_data3=[]
for i in range(0, bits3):
    e1_data3.insert(e1_3 , input3 + 2**i)
    AWE1_3.insert(e1_3, 2**i)
    e1_3 = e1_3 + 1
    e1_data3.insert(e1_3 , input3 - 2**i)
    AWE1_3.insert(e1_3, -2**i)
    e1_3 = e1_3 + 1
    h.insert(i,i+1)

h_awe = [val for num in h for val in (num, -num)]
print("產生1-bit AWE:", AWE1_1)
print("不同輸入資料的AWE是否相等:",AWE1_1 == AWE1_2 == AWE1_3)
print(h_awe)
print(np.size(AWE1_1))

產生1-bit AWE: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472]
不同輸入資料的AWE是否相等: True
[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -20, 21, -21, 22, -22, 23, -23, 24, -24, 25, -25, 26, -26, 27, -27, 28, -28, 29, -29, 30, -30, 31, -31, 32, -32, 33, -33, 34, -34, 35, -35, 36, -36, 37, 

### 產生SEC的模數:

In [11]:
SEC_R = []
AWE1_R= []
for m in range(0,e1_1):
    mod = e1_data1[m] % AN
    SEC_R.append(mod)

for n in range(0,e1_1):
    mod_awe = AWE1_3[n] % AN
    AWE1_R.append(mod_awe)


print("SEC的模數:\n",SEC_R)
print("AWE1的模數:\n",SEC_R)
print("AWE1的模數是否等於SEC模數:", SEC_R == AWE1_R)
print("SEC的大小:",np.size(SEC_R))

SEC的模數:
 [1, 13836, 2, 13835, 4, 13833, 8, 13829, 16, 13821, 32, 13805, 64, 13773, 128, 13709, 256, 13581, 512, 13325, 1024, 12813, 2048, 11789, 4096, 9741, 8192, 5645, 2547, 11290, 5094, 8743, 10188, 3649, 6539, 7298, 13078, 759, 12319, 1518, 10801, 3036, 7765, 6072, 1693, 12144, 3386, 10451, 6772, 7065, 13544, 293, 13251, 586, 12665, 1172, 11493, 2344, 9149, 4688, 4461, 9376, 8922, 4915, 4007, 9830, 8014, 5823, 2191, 11646, 4382, 9455, 8764, 5073, 3691, 10146]
AWE1的模數:
 [1, 13836, 2, 13835, 4, 13833, 8, 13829, 16, 13821, 32, 13805, 64, 13773, 128, 13709, 256, 13581, 512, 13325, 1024, 12813, 2048, 11789, 4096, 9741, 8192, 5645, 2547, 11290, 5094, 8743, 10188, 3649, 6539, 7298, 13078, 759, 12319, 1518, 10801, 3036, 7765, 6072, 1693, 12144, 3386, 10451, 6772, 7065, 13544, 293, 13251, 586, 12665, 1172, 11493, 2344, 9149, 4688, 4461, 9376, 8922, 4915, 4007, 9830, 8014, 5823, 2191, 11646, 4382, 9455, 8764, 5073, 3691, 10146]
AWE1的模數是否等於SEC模數: True
SEC的大小: 76


### 產生2-bit AWE (Arithmetic-Weight Error)

In [12]:
AWE_2=[]
e2=0     ## 錯誤數量
p=0      ## 錯誤位置
e2_data=[]
for i in range(0, bits1):
    for j in range(i+1, bits1):
        e2_data.insert(e2 , (input1 + 2**i + 2**j))
        AWE_2.insert(e2 , (2**i + 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 + 2**i - 2**j))
        AWE_2.insert(e2 , (2**i - 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i + 2**j))
        AWE_2.insert(e2 , (-2**i + 2**j))                
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i -2**j))
        AWE_2.insert(e2 , (-2**i - 2**j))
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        p = p + 1


print(e2)
print(p)
print("產生2-bits AWE:",AWE_2)
print("產生2-bits 錯誤資料:",e2_data)

第 0 筆錯誤資料: 232146323958
第 0 筆算術錯誤: 3
第 1 筆錯誤資料: 232146323954
第 1 筆算術錯誤: -1
第 2 筆錯誤資料: 232146323956
第 2 筆算術錯誤: 1
第 3 筆錯誤資料: 232146323952
第 3 筆算術錯誤: -3
第 4 筆錯誤資料: 232146323960
第 4 筆算術錯誤: 5
第 5 筆錯誤資料: 232146323952
第 5 筆算術錯誤: -3
第 6 筆錯誤資料: 232146323958
第 6 筆算術錯誤: 3
第 7 筆錯誤資料: 232146323950
第 7 筆算術錯誤: -5
第 8 筆錯誤資料: 232146323964
第 8 筆算術錯誤: 9
第 9 筆錯誤資料: 232146323948
第 9 筆算術錯誤: -7
第 10 筆錯誤資料: 232146323962
第 10 筆算術錯誤: 7
第 11 筆錯誤資料: 232146323946
第 11 筆算術錯誤: -9
第 12 筆錯誤資料: 232146323972
第 12 筆算術錯誤: 17
第 13 筆錯誤資料: 232146323940
第 13 筆算術錯誤: -15
第 14 筆錯誤資料: 232146323970
第 14 筆算術錯誤: 15
第 15 筆錯誤資料: 232146323938
第 15 筆算術錯誤: -17
第 16 筆錯誤資料: 232146323988
第 16 筆算術錯誤: 33
第 17 筆錯誤資料: 232146323924
第 17 筆算術錯誤: -31
第 18 筆錯誤資料: 232146323986
第 18 筆算術錯誤: 31
第 19 筆錯誤資料: 232146323922
第 19 筆算術錯誤: -33
第 20 筆錯誤資料: 232146324020
第 20 筆算術錯誤: 65
第 21 筆錯誤資料: 232146323892
第 21 筆算術錯誤: -63
第 22 筆錯誤資料: 232146324018
第 22 筆算術錯誤: 63
第 23 筆錯誤資料: 232146323890
第 23 筆算術錯誤: -65
第 24 筆錯誤資料: 232146324084
第 24 筆算術錯誤: 129
第 25 筆錯誤資料: 2321463

第 357 筆錯誤資料: 232145799671
第 357 筆算術錯誤: -524284
第 358 筆錯誤資料: 232146848239
第 358 筆算術錯誤: 524284
第 359 筆錯誤資料: 232145799663
第 359 筆算術錯誤: -524292
第 360 筆錯誤資料: 232147372535
第 360 筆算術錯誤: 1048580
第 361 筆錯誤資料: 232145275383
第 361 筆算術錯誤: -1048572
第 362 筆錯誤資料: 232147372527
第 362 筆算術錯誤: 1048572
第 363 筆錯誤資料: 232145275375
第 363 筆算術錯誤: -1048580
第 364 筆錯誤資料: 232148421111
第 364 筆算術錯誤: 2097156
第 365 筆錯誤資料: 232144226807
第 365 筆算術錯誤: -2097148
第 366 筆錯誤資料: 232148421103
第 366 筆算術錯誤: 2097148
第 367 筆錯誤資料: 232144226799
第 367 筆算術錯誤: -2097156
第 368 筆錯誤資料: 232150518263
第 368 筆算術錯誤: 4194308
第 369 筆錯誤資料: 232142129655
第 369 筆算術錯誤: -4194300
第 370 筆錯誤資料: 232150518255
第 370 筆算術錯誤: 4194300
第 371 筆錯誤資料: 232142129647
第 371 筆算術錯誤: -4194308
第 372 筆錯誤資料: 232154712567
第 372 筆算術錯誤: 8388612
第 373 筆錯誤資料: 232137935351
第 373 筆算術錯誤: -8388604
第 374 筆錯誤資料: 232154712559
第 374 筆算術錯誤: 8388604
第 375 筆錯誤資料: 232137935343
第 375 筆算術錯誤: -8388612
第 376 筆錯誤資料: 232163101175
第 376 筆算術錯誤: 16777220
第 377 筆錯誤資料: 232129546743
第 377 筆算術錯誤: -16777212
第 3

第 693 筆錯誤資料: 163426847235
第 693 筆算術錯誤: -68719476720
第 694 筆錯誤資料: 300865800675
第 694 筆算術錯誤: 68719476720
第 695 筆錯誤資料: 163426847203
第 695 筆算術錯誤: -68719476752
第 696 筆錯誤資料: 369585277443
第 696 筆算術錯誤: 137438953488
第 697 筆錯誤資料: 94707370499
第 697 筆算術錯誤: -137438953456
第 698 筆錯誤資料: 369585277411
第 698 筆算術錯誤: 137438953456
第 699 筆錯誤資料: 94707370467
第 699 筆算術錯誤: -137438953488
第 700 筆錯誤資料: 232146324051
第 700 筆算術錯誤: 96
第 701 筆錯誤資料: 232146323923
第 701 筆算術錯誤: -32
第 702 筆錯誤資料: 232146323987
第 702 筆算術錯誤: 32
第 703 筆錯誤資料: 232146323859
第 703 筆算術錯誤: -96
第 704 筆錯誤資料: 232146324115
第 704 筆算術錯誤: 160
第 705 筆錯誤資料: 232146323859
第 705 筆算術錯誤: -96
第 706 筆錯誤資料: 232146324051
第 706 筆算術錯誤: 96
第 707 筆錯誤資料: 232146323795
第 707 筆算術錯誤: -160
第 708 筆錯誤資料: 232146324243
第 708 筆算術錯誤: 288
第 709 筆錯誤資料: 232146323731
第 709 筆算術錯誤: -224
第 710 筆錯誤資料: 232146324179
第 710 筆算術錯誤: 224
第 711 筆錯誤資料: 232146323667
第 711 筆算術錯誤: -288
第 712 筆錯誤資料: 232146324499
第 712 筆算術錯誤: 544
第 713 筆錯誤資料: 232146323475
第 713 筆算術錯誤: -480
第 714 筆錯誤資料: 232146324435
第 714 筆算

第 942 筆算術錯誤: 34359738304
第 943 筆錯誤資料: 197786585523
第 943 筆算術錯誤: -34359738432
第 944 筆錯誤資料: 300865800755
第 944 筆算術錯誤: 68719476800
第 945 筆錯誤資料: 163426847283
第 945 筆算術錯誤: -68719476672
第 946 筆錯誤資料: 300865800627
第 946 筆算術錯誤: 68719476672
第 947 筆錯誤資料: 163426847155
第 947 筆算術錯誤: -68719476800
第 948 筆錯誤資料: 369585277491
第 948 筆算術錯誤: 137438953536
第 949 筆錯誤資料: 94707370547
第 949 筆算術錯誤: -137438953408
第 950 筆錯誤資料: 369585277363
第 950 筆算術錯誤: 137438953408
第 951 筆錯誤資料: 94707370419
第 951 筆算術錯誤: -137438953536
第 952 筆錯誤資料: 232146324339
第 952 筆算術錯誤: 384
第 953 筆錯誤資料: 232146323827
第 953 筆算術錯誤: -128
第 954 筆錯誤資料: 232146324083
第 954 筆算術錯誤: 128
第 955 筆錯誤資料: 232146323571
第 955 筆算術錯誤: -384
第 956 筆錯誤資料: 232146324595
第 956 筆算術錯誤: 640
第 957 筆錯誤資料: 232146323571
第 957 筆算術錯誤: -384
第 958 筆錯誤資料: 232146324339
第 958 筆算術錯誤: 384
第 959 筆錯誤資料: 232146323315
第 959 筆算術錯誤: -640
第 960 筆錯誤資料: 232146325107
第 960 筆算術錯誤: 1152
第 961 筆錯誤資料: 232146323059
第 961 筆算術錯誤: -896
第 962 筆錯誤資料: 232146324851
第 962 筆算術錯誤: 896
第 963 筆錯誤資料: 232146322803
第 96

第 1192 筆算術錯誤: 2560
第 1193 筆錯誤資料: 232146322419
第 1193 筆算術錯誤: -1536
第 1194 筆錯誤資料: 232146325491
第 1194 筆算術錯誤: 1536
第 1195 筆錯誤資料: 232146321395
第 1195 筆算術錯誤: -2560
第 1196 筆錯誤資料: 232146328563
第 1196 筆算術錯誤: 4608
第 1197 筆錯誤資料: 232146320371
第 1197 筆算術錯誤: -3584
第 1198 筆錯誤資料: 232146327539
第 1198 筆算術錯誤: 3584
第 1199 筆錯誤資料: 232146319347
第 1199 筆算術錯誤: -4608
第 1200 筆錯誤資料: 232146332659
第 1200 筆算術錯誤: 8704
第 1201 筆錯誤資料: 232146316275
第 1201 筆算術錯誤: -7680
第 1202 筆錯誤資料: 232146331635
第 1202 筆算術錯誤: 7680
第 1203 筆錯誤資料: 232146315251
第 1203 筆算術錯誤: -8704
第 1204 筆錯誤資料: 232146340851
第 1204 筆算術錯誤: 16896
第 1205 筆錯誤資料: 232146308083
第 1205 筆算術錯誤: -15872
第 1206 筆錯誤資料: 232146339827
第 1206 筆算術錯誤: 15872
第 1207 筆錯誤資料: 232146307059
第 1207 筆算術錯誤: -16896
第 1208 筆錯誤資料: 232146357235
第 1208 筆算術錯誤: 33280
第 1209 筆錯誤資料: 232146291699
第 1209 筆算術錯誤: -32256
第 1210 筆錯誤資料: 232146356211
第 1210 筆算術錯誤: 32256
第 1211 筆錯誤資料: 232146290675
第 1211 筆算術錯誤: -33280
第 1212 筆錯誤資料: 232146390003
第 1212 筆算術錯誤: 66048
第 1213 筆錯誤資料: 232146258931
第 1213 筆算術錯誤: -

...

第 1692 筆錯誤資料: 249326201331
第 1692 筆算術錯誤: 17179877376
第 1693 筆錯誤資料: 214966462963
第 1693 筆算術錯誤: -17179860992
第 1694 筆錯誤資料: 249326184947
第 1694 筆算術錯誤: 17179860992
第 1695 筆錯誤資料: 214966446579
第 1695 筆算術錯誤: -17179877376
第 1696 筆錯誤資料: 266506070515
第 1696 筆算術錯誤: 34359746560
第 1697 筆錯誤資料: 197786593779
第 1697 筆算術錯誤: -34359730176
第 1698 筆錯誤資料: 266506054131
第 1698 筆算術錯誤: 34359730176
第 1699 筆錯誤資料: 197786577395
第 1699 筆算術錯誤: -34359746560
第 1700 筆錯誤資料: 300865808883
第 1700 筆算術錯誤: 68719484928
第 1701 筆錯誤資料: 163426855411
第 1701 筆算術錯誤: -68719468544
第 1702 筆錯誤資料: 300865792499
第 1702 筆算術錯誤: 68719468544
第 1703 筆錯誤資料: 163426839027
第 1703 筆算術錯誤: -68719484928
第 1704 筆錯誤資料: 369585285619
第 1704 筆算術錯誤: 137438961664
第 1705 筆錯誤資料: 94707378675
第 1705 筆算術錯誤: -137438945280
第 1706 筆錯誤資料: 369585269235
第 1706 筆算術錯誤: 137438945280
第 1707 筆錯誤資料: 94707362291
第 1707 筆算術錯誤: -137438961664
第 1708 筆錯誤資料: 232146373107
第 1708 筆算術錯誤: 49152
第 1709 筆錯誤資料: 232146307571
第 1709 筆算術錯誤: -16384
第 1710 筆錯誤資料: 232146340339
第 1710 筆算術錯誤: 16384


第 1941 筆算術錯誤: -1073676288
第 1942 筆錯誤資料: 233220000243
第 1942 筆算術錯誤: 1073676288
第 1943 筆錯誤資料: 231072516595
第 1943 筆算術錯誤: -1073807360
第 1944 筆錯誤資料: 234293873139
第 1944 筆算術錯誤: 2147549184
第 1945 筆錯誤資料: 229998905843
第 1945 筆算術錯誤: -2147418112
第 1946 筆錯誤資料: 234293742067
第 1946 筆算術錯誤: 2147418112
第 1947 筆錯誤資料: 229998774771
第 1947 筆算術錯誤: -2147549184
第 1948 筆錯誤資料: 236441356787
第 1948 筆算術錯誤: 4295032832
第 1949 筆錯誤資料: 227851422195
第 1949 筆算術錯誤: -4294901760
第 1950 筆錯誤資料: 236441225715
第 1950 筆算術錯誤: 4294901760
第 1951 筆錯誤資料: 227851291123
第 1951 筆算術錯誤: -4295032832
第 1952 筆錯誤資料: 240736324083
第 1952 筆算術錯誤: 8590000128
第 1953 筆錯誤資料: 223556454899
第 1953 筆算術錯誤: -8589869056
第 1954 筆錯誤資料: 240736193011
第 1954 筆算術錯誤: 8589869056
第 1955 筆錯誤資料: 223556323827
第 1955 筆算術錯誤: -8590000128
第 1956 筆錯誤資料: 249326258675
第 1956 筆算術錯誤: 17179934720
第 1957 筆錯誤資料: 214966520307
第 1957 筆算術錯誤: -17179803648
第 1958 筆錯誤資料: 249326127603
第 1958 筆算術錯誤: 17179803648
第 1959 筆錯誤資料: 214966389235
第 1959 筆算術錯誤: -17179934720
第 1960 筆錯誤資料: 26650612785

第 2191 筆算術錯誤: -34360262656
第 2192 筆錯誤資料: 300866324979
第 2192 筆算術錯誤: 68720001024
第 2193 筆錯誤資料: 163427371507
第 2193 筆算術錯誤: -68718952448
第 2194 筆錯誤資料: 300865276403
第 2194 筆算術錯誤: 68718952448
第 2195 筆錯誤資料: 163426322931
第 2195 筆算術錯誤: -68720001024
第 2196 筆錯誤資料: 369585801715
第 2196 筆算術錯誤: 137439477760
第 2197 筆錯誤資料: 94707894771
第 2197 筆算術錯誤: -137438429184
第 2198 筆錯誤資料: 369584753139
第 2198 筆算術錯誤: 137438429184
第 2199 筆錯誤資料: 94706846195
第 2199 筆算術錯誤: -137439477760
第 2200 筆錯誤資料: 232149469683
第 2200 筆算術錯誤: 3145728
第 2201 筆錯誤資料: 232145275379
第 2201 筆算術錯誤: -1048576
第 2202 筆錯誤資料: 232147372531
第 2202 筆算術錯誤: 1048576
第 2203 筆錯誤資料: 232143178227
第 2203 筆算術錯誤: -3145728
第 2204 筆錯誤資料: 232151566835
第 2204 筆算術錯誤: 5242880
第 2205 筆錯誤資料: 232143178227
第 2205 筆算術錯誤: -3145728
第 2206 筆錯誤資料: 232149469683
第 2206 筆算術錯誤: 3145728
第 2207 筆錯誤資料: 232141081075
第 2207 筆算術錯誤: -5242880
第 2208 筆錯誤資料: 232155761139
第 2208 筆算術錯誤: 9437184
第 2209 筆錯誤資料: 232138983923
第 2209 筆算術錯誤: -7340032
第 2210 筆錯誤資料: 232153663987
第 2210 筆算術錯誤: 7340032

第 2441 筆錯誤資料: 163435235827
第 2441 筆算術錯誤: -68711088128
第 2442 筆錯誤資料: 300857412083
第 2442 筆算術錯誤: 68711088128
第 2443 筆錯誤資料: 163418458611
第 2443 筆算術錯誤: -68727865344
第 2444 筆錯誤資料: 369593666035
第 2444 筆算術錯誤: 137447342080
第 2445 筆錯誤資料: 94715759091
第 2445 筆算術錯誤: -137430564864
第 2446 筆錯誤資料: 369576888819
第 2446 筆算術錯誤: 137430564864
第 2447 筆錯誤資料: 94698981875
第 2447 筆算術錯誤: -137447342080
第 2448 筆錯誤資料: 232196655603
第 2448 筆算術錯誤: 50331648
第 2449 筆錯誤資料: 232129546739
第 2449 筆算術錯誤: -16777216
第 2450 筆錯誤資料: 232163101171
第 2450 筆算術錯誤: 16777216
第 2451 筆錯誤資料: 232095992307
第 2451 筆算術錯誤: -50331648
第 2452 筆錯誤資料: 232230210035
第 2452 筆算術錯誤: 83886080
第 2453 筆錯誤資料: 232095992307
第 2453 筆算術錯誤: -50331648
第 2454 筆錯誤資料: 232196655603
第 2454 筆算術錯誤: 50331648
第 2455 筆錯誤資料: 232062437875
第 2455 筆算術錯誤: -83886080
第 2456 筆錯誤資料: 232297318899
第 2456 筆算術錯誤: 150994944
第 2457 筆錯誤資料: 232028883443
第 2457 筆算術錯誤: -117440512
第 2458 筆錯誤資料: 232263764467
第 2458 筆算術錯誤: 117440512
第 2459 筆錯誤資料: 231995329011
第 2459 筆算術錯誤: -150994944
第 2460 筆錯誤資料:

第 2691 筆錯誤資料: 197249714675
第 2691 筆算術錯誤: -34896609280
第 2692 筆錯誤資料: 301402671603
第 2692 筆算術錯誤: 69256347648
第 2693 筆錯誤資料: 163963718131
第 2693 筆算術錯誤: -68182605824
第 2694 筆錯誤資料: 300328929779
第 2694 筆算術錯誤: 68182605824
第 2695 筆錯誤資料: 162889976307
第 2695 筆算術錯誤: -69256347648
第 2696 筆錯誤資料: 370122148339
第 2696 筆算術錯誤: 137975824384
第 2697 筆錯誤資料: 95244241395
第 2697 筆算術錯誤: -136902082560
第 2698 筆錯誤資料: 369048406515
第 2698 筆算術錯誤: 136902082560
第 2699 筆錯誤資料: 94170499571
第 2699 筆算術錯誤: -137975824384
第 2700 筆錯誤資料: 235367549427
第 2700 筆算術錯誤: 3221225472
第 2701 筆錯誤資料: 231072582131
第 2701 筆算術錯誤: -1073741824
第 2702 筆錯誤資料: 233220065779
第 2702 筆算術錯誤: 1073741824
第 2703 筆錯誤資料: 228925098483
第 2703 筆算術錯誤: -3221225472
第 2704 筆錯誤資料: 237515033075
第 2704 筆算術錯誤: 5368709120
第 2705 筆錯誤資料: 228925098483
第 2705 筆算術錯誤: -3221225472
第 2706 筆錯誤資料: 235367549427
第 2706 筆算術錯誤: 3221225472
第 2707 筆錯誤資料: 226777614835
第 2707 筆算術錯誤: -5368709120
第 2708 筆錯誤資料: 241810000371
第 2708 筆算術錯誤: 9663676416
第 2709 筆錯誤資料: 224630131187
第 2709 筆算術錯誤: -75

In [13]:
DEC_R = []
for m in range(0,e2):
    mod = e2_data[m] % AN
    DEC_R.append(mod)

print("DEC_R的模數:\n",DEC_R)
print(np.size(DEC_R))

DEC_R的模數:
 [3, 13836, 1, 13834, 5, 13834, 3, 13832, 9, 13830, 7, 13828, 17, 13822, 15, 13820, 33, 13806, 31, 13804, 65, 13774, 63, 13772, 129, 13710, 127, 13708, 257, 13582, 255, 13580, 513, 13326, 511, 13324, 1025, 12814, 1023, 12812, 2049, 11790, 2047, 11788, 4097, 9742, 4095, 9740, 8193, 5646, 8191, 5644, 2548, 11291, 2546, 11289, 5095, 8744, 5093, 8742, 10189, 3650, 10187, 3648, 6540, 7299, 6538, 7297, 13079, 760, 13077, 758, 12320, 1519, 12318, 1517, 10802, 3037, 10800, 3035, 7766, 6073, 7764, 6071, 1694, 12145, 1692, 12143, 3387, 10452, 3385, 10450, 6773, 7066, 6771, 7064, 13545, 294, 13543, 292, 13252, 587, 13250, 585, 12666, 1173, 12664, 1171, 11494, 2345, 11492, 2343, 9150, 4689, 9148, 4687, 4462, 9377, 4460, 9375, 8923, 4916, 8921, 4914, 4008, 9831, 4006, 9829, 8015, 5824, 8013, 5822, 2192, 11647, 2190, 11645, 4383, 9456, 4381, 9454, 8765, 5074, 8763, 5072, 3692, 10147, 3690, 10145, 6, 13835, 2, 13831, 10, 13831, 6, 13827, 18, 13823, 14, 13819, 34, 13807, 30, 13803, 66, 13775

In [15]:
total_AWE = AWE1_1 + AWE_2
e_total = e1_data1 + e2_data
R_total = SEC_R + DEC_R
print("還未去除重複error data的數量:",np.size(e_total))
print("還未去除重複餘數的數量:", np.size(R_total))

## 把重複的 AWE 及 error_data 去除
AWE = remove_duplicates(total_AWE)
new_e_total = remove_duplicates(e_total)
new_R_total = remove_duplicates(R_total)
error_total_count = np.size(new_e_total)
print("已去除重複error data的數量:",error_total_count)
print("已未去除重複餘數的數量:",np.size(new_R_total))
print("已未去除重複AWE的數量:",np.size(AWE))
print("所有的AWE為:\n",AWE)
print("所有算術錯誤的資料:\n", new_e_total)

還未去除重複error data的數量: 2888
還未去除重複餘數的數量: 2888
已去除重複error data的數量: 2742
已未去除重複餘數的數量: 2742
已未去除重複AWE的數量: 2742
所有的AWE為:
 [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472, 3, -3, 5, -5, 9, -7, 7, -9, 17, -15, 15, -17, 33, -31, 31, -33, 65, -63, 63, -65, 129, -127, 127, -129, 257, -255, 255, -257, 513, -511, 511, -513, 1025, -1023, 1023, -1025, 2049, -2047, 2047, -2049, 4097, -4095, 4095, -4097, 819

### 所有AWE數量: $4*C^{bits}_{2}$ $-$ $2*(bits-1)$ - $2*(bits-2)$
* $2*(bits-1)$: 所有的 single AWE都可以分解成已重複的 double AWE, 除了MSB例外
* EX: $(+,0)$ 只能分解成 $(0,++)$ , $(-,0)$ 只能分解成 $(0,--)$
* $2*(bits-2)$: double AWE重複的個數
* EX: $(+,0,-)$ = $(0,+,+)$ and $(-,0,+)$ = $(0,-,-)$

### Trade-off Algorithm: 2025/05/27 使用 Vivado HLX IP 驗證
* 此解出來的數字為位置及正負號 EX: $-1$ 代表 $-2^0$ , $-5$ 代表 $-2^4$ 以此類推

In [17]:
import math
import time
### 給定一個DEC_R, 找錯誤位置h1及h2(包含錯誤正負號)
import numpy as np

h1 = 0; h2 = 0; h_temp = 0
possible_h1 = []; possible_h2 = []; possible_AWE = []
output = []; possible_SE_h1=[]

start_time = time.time()  # 記錄開始時間
for R, error in zip(new_R_total, new_e_total): 
    found = False                                       # 旗標，用來標示是否要跳出 for H 循環 
    for H in range(0,bits1):                              
        for s in range(-1,2,2):                       
            h1 = s*(H+1)
            ip.write_mm(0, h1)
            R1 = ip.read_mm(4)
            R2 = R - R1
            if R2 < 0:
                R2 = AN + R2
            
            ip.write_mm(8, R2)
            h2 = twos_complement_32bit(ip.read_mm(12))                  
            if (h2 != 0):
                s2 = get_sign(h2)                     
                possible_h1.append(h1)                  # 若找不到對應的h2,h2定為0(函數內容已定義)
                possible_h2.append(h2)                  # 代表錯誤可能是三個或以上(multiple errors)
                output.append((error - (s*2**(abs(h1)-1)) - (s2 * 2**(abs(h2)-1))) // AN)
                found = True
                break
        if found:
            break
            
end_time = time.time()  # 記錄結束時間
elapsed_time = end_time - start_time  # 計算執行時間
print(f"執行時間: {elapsed_time:.6f} 秒")
print("更正後的值是否和原data相同:", data1 == output.pop())
print(elapsed_time / np.size(possible_h1))
##print(possible_h1)
##print(possible_h2)
##print("更正後的值:",output)
print(np.size(possible_h1))
print(np.size(possible_h2))
##print("輸入的 error data的總數是否和經過演算法後相同:", error_total_count == np.size(possible_h1))
##print(possible_SE_h1)

執行時間: 7.708428 秒
更正後的值是否和原data相同: True
0.002811242854134346
2742
2742
